# 简介

无需多说，一路运行即可！

此脚本由**[托尼是塔克](https://wxp123.me)**创建，仅用于学习，请勿滥用。如有疑问，拉到最后！

In [ ]:
#@title 查看分配到的设备
# 查看设备，是K80还是T4,如果是K80...！
! /opt/bin/nvidia-smi

# 第一步 准备好workspace
这一步你可以选着两种方式
1. 使用默认的workspace，你无需自己上传，仅用于熟悉操作。
2. 通过Google Drive （谷歌云盘）上传自己的workspace到指定目录。



谷歌云盘地址：https://drive.google.com/drive/my-drive  
注意：谷歌网址现在都需要“科学上网”才能访问。  

In [ ]:
#@title 开始挂载 
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#创建DeepFaceLab目录，并且进入目录
%cd /content/drive/My Drive/
!mkdir DeepFaceLab
%cd /content/drive/My Drive/DeepFaceLab/


这一环节有两个选择：  
如果你第一次使用，可以加载示例项目，然后往后运行。  
如果你是二次运行，就不需要加载了。  
如果你希望上传本地素材，可以先用pack脚本把素材打包成faceset.pak到aligned目录，无须解压，可直接进行训练和转换，加载速度会很快。


In [ ]:
#@title 加载示例项目workspace 
# 1. 练手可以使用这一行，直接git clone一个workspace
%cd /content/drive/My Drive/DeepFaceLab/
!rm -rf workspace
!git clone https://github.com/dream80/DFLWorkspace.git workspace

#第二步 安装DeepFaceLab
获取源代码，安装依赖,根据自己的情况选择版本  

last:最新版  

In [ ]:
#@title 开始安装

Version = "last" #@param ["v2.02.03", "v2.02.23b", "v2.02.28","v2.03.07","v2.03.15","v2.06.01","v2.06.19","last"]
%cd /content/drive/My Drive/DeepFaceLab
!rm -fr DeepFaceLab_Colab
if Version=="620":
  print("620版加载中....")
  # 获取DFL源代码v1.6.1稳定版
  !git clone -b v1.6.1 https://github.com/dream80/DeepFaceLab_Colab.git
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt  
elif Version=="last":
  print("最新版加载中....")
  !git clone https://github.com/iperov/DeepFaceLab.git  DeepFaceLab_Colab
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt
else:
  print(Version+"加载中....")
  cmd="clone -b "+Version+" https://github.com/dream80/DeepFaceLab_Colab.git"  
  !git $cmd
  %cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  !pip install -r requirements-colab.txt

!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0


# 第三步. 提取脸部

这个步骤其实是分两次的。第一次选src，第二次选dst，很多对Deepfacelab不了解的人可能只点了一个，后面就会报错了。  
  
这一步，其实有三个小步骤，分解视频，提取人脸，打包素材。 如果有需要可以将他们拆分开来。一般是不用这么做的。 





In [ ]:
#@title 开始提取
target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/
  !python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
  !python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/
  !python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --output-debug
  !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset


In [ ]:
#@title 开始排序
target = "src" #@param ["src","dst"]

if target=="src":
  #Src排序，可以通过谷歌云盘查看结果，删除不良图片
  cmd = "main.py sort --input-dir ../workspace/data_src/aligned"
  
else:
  cmd = "main.py sort --input-dir ../workspace/data_dst/aligned "

!python $cmd

# 第四步. 训练模型


*  支持SAEHD,Quick96等模型，根据自己的情况选择模型。
*  第一次需要配置模型参数，不懂的直接回车默认。
*  不想训练了可以点击停止，停止时会抛出异常，但是没什么关系。下次可以继续训练

In [ ]:
#@title 开始训练
Model = "SAEHD" #@param ["SAEHD","Quick96"]

%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/"
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

**模型预览方法请参考：**
https://www.wxp123.me/p/329

简单描述：左侧->文件->/driver/..../model/ *preview*.jpg->双击  
  
这种方法比在脚本中预览和在网盘预览都要方便，并且无需启动预览配置！

# 第五步. 转换输出

训练模型的时候选了什么，这里就选什么  
比如训练H128,这里就选H128


In [ ]:
#@title 开始转换图片

Model = "SAEHD" #@param ["SAEHD", "Quick96"]
%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab/"
cmd = " main.py merge --output-mask-dir ../workspace/data_dst/merged_mask --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model
!python $cmd

In [ ]:
#@title 开始图片转视频
!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4 --include-audio
!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged_mask --output-file ../workspace/result_mask.mp4 --reference-file ../workspace/data_dst.mp4 --include-audio --lossless

#第六步. 继续训练
当你第二次开始训练，或者掉线之后继续训练时并不需要执行上面所有的步骤。只需要下面简单的几个步骤。
1. 挂载云盘
2. 安装依赖
3. 开始训练  


In [ ]:
#@title 一键运行
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

Model = "SAEHD" #@param ["SAEHD" , "Quick96"]


from google.colab import drive
drive.mount('/content/drive', force_remount=True)


# 进入DeepFaceLab_Colab目录
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

# 安装Python依赖
!pip install -r requirements-colab.txt
!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0

# 开始训练SAE ，如果是其他模型，修改后面的参数即可，比如H128。
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd 

Mounted at /content/drive
/content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
  Using cached https://files.pythonhosted.org/packages/24/06/d560630eb9e36d90d69fe57d9ff762d8f501664ce478b8a0ae132b3c3008/scikit_image-0.14.2-cp36-cp36m-manylinux1_x86_64.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scikit-image 0.17.2
    Uninstalling scikit-image-0.17.2:
      Successfully uninstalled scikit-image-0.17.2
  Using cached https://files.pythonhosted.org/packages/0e/ba/53e1bfbdfd0f94514d71502e3acea494a8b4b57c457adbc333ef386485da/scikit_image-0.17.2-cp36-cp36m-manylinux1_x86_64.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scikit-image 0.14.2
    Uninstalling scikit-image-0.14.2:
      Successfully uninstalled scikit-image-0.14.2
Reading package lists... Done
Building dependency tr

# 工具



打包素材。可以加快上传，下载，加载的速度。

In [ ]:
#@title 素材打包

target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
  !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset
 
  

In [ ]:
#@title 素材解包

target = "src" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned --unpack-faceset
else:
  !python main.py util --input-dir ../workspace/data_dst/aligned --unpack-faceset
 
  

In [ ]:
#@title 素材增强

target = "src" #@param ["src"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab
if target=="src":
  !python main.py facesettool enhance --input-dir ../workspace/data_src/aligned
else:
  !python main.py facesettool enhance --input-dir ../workspace/data_dst/aligned
 
  

# 其他

 谷歌云地址：https://drive.google.com/drive/my-drive

 作者邮箱 ：wpgdream@gmail.com
 
 使用教程：https://www.wxp123.me
 
 
不求别的，只求在github给个小星星^_^

https://github.com/dream80/DeepFaceLab_Colab

右上角star 谢谢!!!